In [8]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Loding libray and settingp config 

# Connecting with CCF netowrks.

# Loading libaries 
import requests
import json
import os
import subprocess
import sys
import base64
import hashlib
from urllib.parse import urlparse
import subprocess
import json
import time
import os
import glob


RSA_SIZE = 2048


DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]

DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"

RSA_SIZE = 2048
# CCF network node
server="https://127.0.0.1:8000"

num_users = 4

In [12]:
# Getting Network metrices
url = server + "/app/api/metrics"

try:
    response = requests.get(url, verify='service_cert.pem')

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)


Status Code: 200

Response Headers:
content-length: 176
content-type: application/json
x-ms-ccf-transaction-id: 2.12

Response Body:
{
    "metrics": [
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "api/metrics",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "commit",
            "retries": 0
        }
    ]
}


In [13]:

#  delting file if available for user 
def delete_user_files(num_users):
    for i in range(num_users):
        # File names based on user index
        cert_file = f"user{i}_cert.pem"
        priv_key_file = f"user{i}_privk.pem"
        enc_priv_file = f"user{i}_enc_privk.pem"
        enc_pub_file = f"user{i}_enc_pubk.pem"

        # Delete the files if they exist
        for file in [cert_file, priv_key_file, enc_priv_file, enc_pub_file]:
            if os.path.exists(file):
                os.remove(file)
                print(f"Deleted {file}")
            else:
                print(f"{file} does not exist")

        # Delete all JSON files related to the user
        for json_file in glob.glob(f"set_user{i}*.json"):
            os.remove(json_file)
            print(f"Deleted {json_file}")

# Specify the number of users to delete

delete_user_files(num_users)


user0_cert.pem does not exist
user0_privk.pem does not exist
user0_enc_privk.pem does not exist
user0_enc_pubk.pem does not exist
user1_cert.pem does not exist
user1_privk.pem does not exist
user1_enc_privk.pem does not exist
user1_enc_pubk.pem does not exist
user2_cert.pem does not exist
user2_privk.pem does not exist
user2_enc_privk.pem does not exist
user2_enc_pubk.pem does not exist
user3_cert.pem does not exist
user3_privk.pem does not exist
user3_enc_privk.pem does not exist
user3_enc_pubk.pem does not exist


In [14]:
# Keygenrator function for public and private key with ballat request file


def generate_keys(name, curve=DEFAULT_CURVE, generate_encryption_key=False):
    if not name:
        print("Error: The name of the participant should be specified (e.g. member0 or user1)")
        sys.exit(1)

    if curve not in SUPPORTED_CURVES:
        print(f"{curve} curve is not in {SUPPORTED_CURVES}")
        sys.exit(1)

    digest = DIGEST_SHA384 if curve == DEFAULT_CURVE else DIGEST_SHA256

    cert = f"{name}_cert.pem"
    privk = f"{name}_privk.pem"

    print(f"-- Generating identity private key and certificate for participant \"{name}\"...")
    print(f"Identity curve: {curve}")

    subprocess.run(["openssl", "ecparam", "-out", privk, "-name", curve, "-genkey"], check=True)
    subprocess.run(["openssl", "req", "-new", "-key", privk, "-x509", "-nodes", "-days", "365", "-out", cert, f"-{digest}", "-subj", f"/CN={name}"], check=True)

    print(f"Identity private key generated at: {privk}")
    print(f"Identity certificate generated at: {cert}")

    enc_priv, enc_pub = None, None
    if generate_encryption_key:
        print(f"-- Generating RSA encryption key pair for participant \"{name}\"...")

        enc_priv = f"{name}_enc_privk.pem"
        enc_pub = f"{name}_enc_pubk.pem"

        subprocess.run(["openssl", "genrsa", "-out", enc_priv, str(RSA_SIZE)], check=True)
        subprocess.run(["openssl", "rsa", "-in", enc_priv, "-pubout", "-out", enc_pub], check=True)

        print(f"Encryption private key generated at: {enc_priv}")
        print(f"Encryption public key generated at: {enc_pub}")

    return cert, enc_pub



# # CA certificate creator function
# def create_certificate(cert_name):
#     cert_file = f"{cert_name}_cert.pem"
#     set_user_file = f"set_{cert_name}.json"
    
#     # Call the generate_keys function (make sure to include it in your script)
#     generate_keys(cert_name)

#     # Read the certificate file and format it
#     with open(cert_file, 'r') as file:
#         cert_content = file.read().replace('\n', '\n')

#     # Create the JSON content
#     user_json = {
#         "actions": [
#             {
#                 "name": "set_user",
#                 "args": {
#                     "cert": cert_content
#                 }
#             }
#         ]
#     }

#     # Write the JSON to a file
#     with open(set_user_file, 'w') as file:
#         json.dump(user_json, file, indent=2)
#     print(f"JSON file created at: {set_user_file}")
    

def create_certificatefor_member(cert_name):
    cert_file = f"{cert_name}_cert.pem"
    set_member_file = f"set_{cert_name}.json"
    
    # Call the generate_keys function (make sure to include it in your script)
    cert_content, pub_key_content = generate_keys(cert_name)

    # Read the certificate file and format it
    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\n')

    # Create the JSON content
    user_json = {
        "actions": [
            {
                "name": "set_member",
                "args": {
                    "cert": cert_content,
                    "encryption_pub_key": pub_key_content
                }
            }
        ]
    }

    # Write the JSON to a file
    with open(set_member_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_member_file}")
    


def send_secure_request(url, request_data_path, signing_privk, signing_cert, command="post"):
    def read_request_data(request_path):
        if request_path.startswith('@'):
            with open(request_path[1:], 'r') as file:
                return file.read()
        else:
            return request_path

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(method, url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): {method.lower()} {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    request_data = read_request_data(request_data_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(command, url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk)
    key_id = get_cert_key_id(signing_cert)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"'
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.text




# sending voting to add  the user based on proposal id 

def send_ballot_request(server_url, proposal_id, data_file_path, signing_privk_path, signing_cert_path):
    def read_request_data(file_path):
        with open(file_path, 'r') as file:
            return file.read()

    def calculate_digest(data):
        sha256_hash = hashlib.sha256()
        sha256_hash.update(data.encode('utf-8'))
        return base64.b64encode(sha256_hash.digest()).decode()

    def create_signature(string_to_sign, priv_key_path):
        process = subprocess.Popen(
            ['openssl', 'dgst', '-sha384', '-sign', priv_key_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        signature, _ = process.communicate(string_to_sign.encode())
        return base64.b64encode(signature).decode().replace('\n', '')

    def prepare_string_to_sign(url, digest, data_length):
        parsed_url = urlparse(url)
        path = parsed_url.path
        return f"(request-target): post {path}\ndigest: SHA-256={digest}\ncontent-length: {data_length}"

    def get_cert_key_id(cert_path):
        proc = subprocess.Popen(
            ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, _ = proc.communicate()
        fingerprint = output.decode().split('=')[1].replace(':', '').lower().replace('\n', '')
        return fingerprint

    url = f"{server_url}/gov/proposals/{proposal_id}/ballots"
    request_data = read_request_data(data_file_path)
    request_digest = calculate_digest(request_data)
    string_to_sign = prepare_string_to_sign(url, request_digest, str(len(request_data)))
    signature = create_signature(string_to_sign, signing_privk_path)
    key_id = get_cert_key_id(signing_cert_path)

    headers = {
        "Digest": f"SHA-256={request_digest}",
        "Authorization": f'Signature keyId="{key_id}",algorithm="hs2019",headers="(request-target) digest content-length",signature="{signature}"',
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=request_data, headers=headers, verify='service_cert.pem')
    return response.status_code, response.json()





# creation of user id based on certifacte 
def get_certificate_fingerprint(cert_path):
    process = subprocess.Popen(
        ['openssl', 'x509', '-in', cert_path, '-noout', '-fingerprint', '-sha256'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    output, _ = process.communicate()
    fingerprint = output.decode().split('=')[1].replace(':', '').lower().strip()
    return fingerprint



def create_certificate(cert_name, entity_type):
    cert_file = f"{cert_name}_cert.pem"
    set_file = f"set_{cert_name}.json"
    
    # Adjust the generate_keys function call based on entity type
    if entity_type == "member":
        cert_content, pub_key_content = generate_keys(cert_name, generate_encryption_key=True)
    elif entity_type == "user":
        cert_content = generate_keys(cert_name, generate_encryption_key=False)[0]
    else:
        raise ValueError("entity_type must be 'user' or 'member'")

    # Read the certificate file and format it
    with open(cert_file, 'r') as file:
        cert_content = file.read().replace('\n', '\n')

    # Create the JSON content
    action_args = {"cert": cert_content}
    if entity_type == "member":
        action_args["encryption_pub_key"] = pub_key_content

    user_json = {
        "actions": [
            {
                "name": f"set_{entity_type}",
                "args": action_args
            }
        ]
    }

    # Write the JSON to a file
    with open(set_file, 'w') as file:
        json.dump(user_json, file, indent=2)
    print(f"JSON file created at: {set_file}")



In [15]:
import subprocess
import json
import os

def sign_state_digest(request_json_path, priv_key_path, output_path):
    """
    Reads a state digest from a JSON file, signs it, and saves the signed digest.

    :param request_json_path: Path to the JSON file containing the state digest.
    :param priv_key_path: Path to the private key file for signing.
    :param output_path: Path to save the signed digest.
    """
    # Step 1: Read the state digest from request.json
    with open(request_json_path, 'r') as file:
        data = json.load(file)
        state_digest = data.get('state_digest')

    if not state_digest:
        print("State digest not found in the file.")
        return

    # Step 2: Sign the digest
    # This is a placeholder for the ccf_cose_sign1 utility command
    # Replace this subprocess call with the actual command you need to run
    command = [
        'ccf_cose_sign1',
        '--ccf-gov-msg-type', 'ack',
        '--ccf-gov-msg-created_at', os.popen('date -uIs').read().strip(),
        '--signing-key', priv_key_path,
        '--signing-cert', priv_key_path,  # If a separate cert is required, replace this
        '--content', request_json_path
    ]

    with open(output_path, 'w') as outfile:
        subprocess.run(command, stdout=outfile)

    print(f"Signed state digest saved to {output_path}")





In [18]:
# Add member 
# Example usage
# Create member1
create_certificate("member1", "member")

# Create member2
create_certificate("member2", "member")


-- Generating identity private key and certificate for participant "member1"...
Identity curve: secp384r1
Identity private key generated at: member1_privk.pem
Identity certificate generated at: member1_cert.pem
-- Generating RSA encryption key pair for participant "member1"...
Encryption private key generated at: member1_enc_privk.pem
Encryption public key generated at: member1_enc_pubk.pem
JSON file created at: set_member1.json
-- Generating identity private key and certificate for participant "member2"...
Identity curve: secp384r1
Identity private key generated at: member2_privk.pem
Identity certificate generated at: member2_cert.pem
-- Generating RSA encryption key pair for participant "member2"...


writing RSA key


Encryption private key generated at: member2_enc_privk.pem
Encryption public key generated at: member2_enc_pubk.pem
JSON file created at: set_member2.json


writing RSA key


In [19]:

# Adding new two member and then activationg

member1 = "member1"
member2 = "member2"

# Send request to add member1
status_code, response_json = send_secure_request(
    server + "/gov/proposals",
    f"@set_{member2}.json",
    "member0_privk.pem",
    "member0_cert.pem",
    "post"
)
print("Status code:", status_code, "Response:", response_json)
response=json.loads(response_json)
# Check if the request was successful and then proceed to vote
if status_code == 200 and response["state"]=="open" :
    # Extract the proposal ID from response_json, replace 'proposal_id_field' with the actual field name
    proposal_id=response['proposal_id']

    # Send a vote for the proposal
    vote_status, response_json = send_ballot_request(
        server,
        proposal_id,
        "vote_accept.json",
        "member0_privk.pem",
        "member0_cert.pem"
    )
    print("Voting status:", vote_status, "Response:", response_json)
elif status_code==200 and response["state"]=="Accepted":
    print("propsal is already accpeted")
else:
    print("Failed to add member, status code:", status_code)
  


Status code: 200 Response: {"ballot_count":0,"proposal_id":"53931f5cf842224ddc08ff850b6ab737ae115dfab998b427faa4e92924a82064","proposer_id":"ba35cad897341b916c283797f8996ed3c4b4bea965abc586aa6c758af6da37c5","state":"Accepted","votes":{}}
propsal is already accpeted


In [57]:
%pip install ccf

  Using cached cryptography-41.0.5-cp37-abi3-manylinux_2_28_x86_64.whl.metadata (5.2 kB)
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached pycose-1.0.1-py3-none-any.whl (48 kB)
  Using cached string_color-1.2.3-py3-none-any.whl (13 kB)
  Using cached cffi-1.16.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached cbor2-5.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
  Using cached ecdsa-0.18.0-py2.py3-none-any.whl (142 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached certvalidator-0.11.1-py2.py3-none-any.whl (31 kB)
  Using cached Columnar-1.4.1-py3-none-any.whl (11 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
  Using cached oscrypto-1.3.0-py2.py3-none-any.whl (194 kB)
Usin

In [23]:
memer_id=get_certificate_fingerprint("member1_cert.pem")
print("memberid",memberid)

memberid ff2bb1adce8c2d9bbe8226fd29f1699c137df3b8b881943aff0acdec15a9df5c


In [25]:
import requests
import json
import subprocess
from datetime import datetime

# Replace these with appropriate values
ccf_node_address = "https://127.0.0.1:8000"
member_id = memer_id
api_version = "2023-06-01-preview"
service_cert_path = "service_cert.pem"
new_member_privk_path = "member1_privk.pem"
new_member_cert_path = "member1_cert.pem"

# Update and retrieve the latest state digest
update_url = f"{ccf_node_address}/gov/members/state-digests/{member_id}:update"
response = requests.post(update_url, cert=(new_member_cert_path, new_member_privk_path), verify=service_cert_path)
state_digest = response.json()
print("state_digest",state_digest)

# Save state digest to a file
with open("request.json", "w") as file:
    json.dump(state_digest, file)

# Prepare ccf_cose_sign1 command
sign1_cmd = [
    "ccf_cose_sign1",
    "--ccf-gov-msg-type", "ack",
    "--ccf-gov-msg-created_at", datetime.utcnow().isoformat(),
    "--signing-key", new_member_privk_path,
    "--signing-cert", new_member_cert_path,
    "--content", "request.json"
]

# Execute ccf_cose_sign1 and capture the output
signed_digest = subprocess.run(sign1_cmd, capture_output=True, text=True).stdout

# Acknowledge the state digest
ack_url = f"{ccf_node_address}/gov/members/state-digests/{member_id}:ack?api-version={api_version}"
headers = {"content-type": "application/cose"}
ack_response = requests.post(ack_url, data=signed_digest, headers=headers, cert=(new_member_cert_path, new_member_privk_path), verify=service_cert_path)

# Check response
if ack_response.ok:
    print("State digest acknowledged successfully.")
else:
    print("Failed to acknowledge state digest.")


state_digest {'error': {'code': 'ResourceNotFound', 'message': 'Unknown path: /members/state-digests/4117e4077f39b32efad85887107306d776b971177becf7723386def5c3c03a96:update.'}}


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 4: invalid start byte

In [42]:
def main_process(num_initial_users):
    # Generate keys and certificates for all users
    for i in range(num_initial_users):
        create_certificate(f"user{i}","user")

    # Submit proposal and voting for each user
    for i in range(num_initial_users):
        user_name = f"user{i}"
        user_cert_path = f"{user_name}_cert.pem"
        user_key_path = f"{user_name}_privk.pem"

        # Member0 submits the proposal
        status_code, response_json = send_secure_request(server + "/gov/proposals", f"@set_{user_name}.json", "member0_privk.pem", "member0_cert.pem", "post")
        print("proposal_id",json.loads(response_json)['proposal_id'])
        proposal_id=json.loads(response_json)['proposal_id']

        # Other members vote
        for j in range(1, 3):  # Assuming 2 other members (member1 and member2)
            vote_status, response_json = send_ballot_request(server, proposal_id, "vote_accept.json", f"member{j}_privk.pem", f"member{j}_cert.pem")
            # print("vote_status",vote_status,"response_json",response_json)
            if vote_status != 200:
                print(f"Voting failed for {user_name} by member{j}")
                continue
            time.sleep(1)  # Delay for processing

        # Create account for the user
        user_id=get_certificate_fingerprint(user_cert_path)
        print("Actual id",user_id) 
   
      
       


# Run the script for a specified number of users
main_process(num_users)

-- Generating identity private key and certificate for participant "user0"...
Identity curve: secp384r1
Identity private key generated at: user0_privk.pem
Identity certificate generated at: user0_cert.pem
JSON file created at: set_user0.json
-- Generating identity private key and certificate for participant "user1"...
Identity curve: secp384r1
Identity private key generated at: user1_privk.pem
Identity certificate generated at: user1_cert.pem
JSON file created at: set_user1.json
-- Generating identity private key and certificate for participant "user2"...
Identity curve: secp384r1
Identity private key generated at: user2_privk.pem
Identity certificate generated at: user2_cert.pem
JSON file created at: set_user2.json
-- Generating identity private key and certificate for participant "user3"...
Identity curve: secp384r1
Identity private key generated at: user3_privk.pem
Identity certificate generated at: user3_cert.pem
JSON file created at: set_user3.json
proposal_id 57bf8ac0448aefd6f8dc